In [77]:
import openpyxl
import numpy as np
from openpyxl.cell import get_column_letter, column_index_from_string

#### Create dict object to match different reviewers with their score

In [256]:
# There are 5 different reviewers for each proposal, the same reviewer can 
# appear in multiple proposals under different reviewer number. This 
# function looks through each of the reviewer columns and creates a mapping 
# of a reviewer column and which proposal row it belongs to.

def getRowColDictFromName(name):
    rowColDict = {}
    for colNum in range(column_index_from_string("AG"), column_index_from_string("AL")):
        for rowNum in range(2, dataSheet.max_row+1):
            if (dataSheet.cell(row=rowNum, column=colNum).value == name):
                rowColDict[rowNum] = colNum
    return rowColDict

#### Determine reviewers particular review column based on their review location

In [80]:
def getMeritScoreColumn(nameColumn):
    if(nameColumn == 33):
            return "G"
    elif(nameColumn == 34):
            return "H"
    elif(nameColumn == 35):
            return "I"
    elif(nameColumn == 36):
            return "J"
    elif(nameColumn == 37):
            return "K"
        
def getSuitScoreColumn(nameColumn):
    if(nameColumn == 33):
            return "O"
    elif(nameColumn == 34):
            return "P"
    elif(nameColumn == 35):
            return "Q"
    elif(nameColumn == 36):
            return "R"
    elif(nameColumn == 37):
            return "S"        
        
def getCapScoreColumn(nameColumn):
    if(nameColumn == 33):
            return "W"
    elif(nameColumn == 34):
            return "X"
    elif(nameColumn == 35):
            return "Y"
    elif(nameColumn == 36):
            return "Z"
    elif(nameColumn == 37):
            return "AA"   

#### Report generator

In [253]:
# Run through list of names, for each name create a dict map of their particular
# review column for a particular propsal row.  Grab all the data from that row and
# certain columns, do calculations and write to sheet.

def generateReport():

    for r in xrange(2, nameSheet.max_row+1):    
        
        # Read name and email address off nameSheet workbook
        name = nameSheet.cell(row=r, column=1).value
        email = nameSheet.cell(row=r, column=4).value
        # Create a dict map for each name
        nameDict = getRowColDictFromName(name)
        
        # Determine current row in report, so not to overwrite our work.
        currentRow = reportSheet.max_row+1
        # Multiple reviews per singular reviewer will be generated outward to the right
        # Using a column offset*column to create groups of reviews.
        columnOffset = 1
        
        
        # Fill name and email into report, only do this once per viewer
        reportSheet.cell(row=currentRow, column=1).value = name
        reportSheet.cell(row=currentRow, column=2).value = email

        reportSheet.cell(row=1, column=1).value = "Reviewer Name"
        reportSheet.cell(row=1, column=2).value = "Email"
        
        for x in nameDict:
            
            #Create offset columns indices 
            if(columnOffset == 1):
                colOffset = (columnOffset - 1) * 14 
            elif(columnOffset > 1):
                colOffset = (columnOffset - 1) * 12 
          
            # Generate new headers for each propsal review
            reportSheet.cell(row=1, column=3+colOffset).value = "Cycle"+str(columnOffset)
            reportSheet.cell(row=1, column=4+colOffset).value = "Proposal ID"+str(columnOffset)
            reportSheet.cell(row=1, column=5+colOffset).value = "Proposal Title"+str(columnOffset)
            reportSheet.cell(row=1, column=6+colOffset).value = "Beamline"+str(columnOffset)
            reportSheet.cell(row=1, column=7+colOffset).value = "Merit"+str(columnOffset)
            reportSheet.cell(row=1, column=8+colOffset).value = "Suit"+str(columnOffset)
            reportSheet.cell(row=1, column=9+colOffset).value = "Cap"+str(columnOffset)
            reportSheet.cell(row=1, column=10+colOffset).value = "Merit Final"+str(columnOffset)
            reportSheet.cell(row=1, column=11+colOffset).value = "Merit SD"+str(columnOffset)
            reportSheet.cell(row=1, column=12+colOffset).value = "Suit Final"+str(columnOffset)
            reportSheet.cell(row=1, column=13+colOffset).value = "Cap Final"+str(columnOffset)
            reportSheet.cell(row=1, column=14+colOffset).value = "Alloc"+str(columnOffset)      
            
                        
            #Determin particular reviewer score column based on which review number they were for that propsal
            meritCol = column_index_from_string(getMeritScoreColumn(nameDict[x]))
            suitCol = column_index_from_string(getSuitScoreColumn(nameDict[x]))
            capCol = column_index_from_string(getCapScoreColumn(nameDict[x]))

            reportSheet.cell(row=currentRow, column=3+colOffset).value = dataSheet.cell(row=x, column=column_index_from_string("A")).value
            reportSheet.cell(row=currentRow, column=4+colOffset).value = dataSheet.cell(row=x, column=column_index_from_string("B")).value
            reportSheet.cell(row=currentRow, column=5+colOffset).value = dataSheet.cell(row=x, column=column_index_from_string("E")).value
            reportSheet.cell(row=currentRow, column=6+colOffset).value = dataSheet.cell(row=x, column=column_index_from_string("D")).value
            reportSheet.cell(row=currentRow, column=7+colOffset).value = dataSheet.cell(row=x, column=meritCol).value
            reportSheet.cell(row=currentRow, column=8+colOffset).value = dataSheet.cell(row=x, column=suitCol).value
            reportSheet.cell(row=currentRow, column=9+colOffset).value = dataSheet.cell(row=x, column=capCol).value
           

            # Merit Final scores. It may not be the case that there were 6 reviewers for the proposal.
            # A blank score isn't a score of zero. Value will be checked against this as to not interfer
            # with calculation.
            m0 = (dataSheet.cell(row=x, column=6).value)
            m1 = (dataSheet.cell(row=x, column=7).value)
            m2 = (dataSheet.cell(row=x, column=8).value)
            m3 = (dataSheet.cell(row=x, column=9).value)
            m4 = (dataSheet.cell(row=x, column=10).value)
            m5 = (dataSheet.cell(row=x, column=11).value)

            mData = []
            
            if(m0):
                mData.append(m0)
            if(m1):
                mData.append(m1)
            if(m2):
                mData.append(m2)
            if(m3):
                mData.append(m3)
            if(m4):
                mData.append(m4)
            if(m5):
                mData.append(m5)    

            mFinal = np.average(mData)
            reportSheet.cell(row=currentRow, column=10+colOffset).value = np.round(mFinal,2)

            # Merit Standard Deviation, rounded to 2 decimal places.
            mStd = np.std(mData)
            reportSheet.cell(row=currentRow, column=11+colOffset).value = np.round(mStd, 2)

            # Suit Final scores. It may not be the case that there were 6 reviewers for the proposal.
            # A blank score isn't a score of zero. Value will be checked against this as to not interfer
            # with calculation.
            s0 = (dataSheet.cell(row=x, column=14).value)
            s1 = (dataSheet.cell(row=x, column=15).value)
            s2 = (dataSheet.cell(row=x, column=16).value)
            s3 = (dataSheet.cell(row=x, column=17).value)
            s4 = (dataSheet.cell(row=x, column=18).value)
            s5 = (dataSheet.cell(row=x, column=19).value)

            sData = []
            if(s0):
                sData.append(s0)
            if(s1):
                sData.append(s1)
            if(s2):
                sData.append(s2)
            if(s3):
                sData.append(s3)
            if(s4):
                sData.append(s4)
            if(s5):
                sData.append(s5)    

            sFinal = np.average(sData)
            reportSheet.cell(row=currentRow, column=12+colOffset).value = np.round(sFinal,2)

            # Cap Final scores. It may not be the case that there were 6 reviewers for the proposal.
            ## A blank score isn't a score of zero. Value will be checked against this as to not interfer
            # with calculation.
            c0 = (dataSheet.cell(row=x, column=22).value)
            c1 = (dataSheet.cell(row=x, column=23).value)
            c2 = (dataSheet.cell(row=x, column=24).value)
            c3 = (dataSheet.cell(row=x, column=25).value)
            c4 = (dataSheet.cell(row=x, column=26).value)
            c5 = (dataSheet.cell(row=x, column=27).value)

            cData = []
            if(c0):
                cData.append(c0)
            if(c1):
                cData.append(c1)
            if(c2):
                cData.append(c2)
            if(c3):
                cData.append(c3)
            if(c4):
                cData.append(c4)
            if(c5):
                cData.append(c5)

            cFinal = np.average(cData)
            reportSheet.cell(row=currentRow, column=13+colOffset).value = np.round(cFinal,2)

            # Allocation value
            reportSheet.cell(row=currentRow, column=14+colOffset).value = dataSheet.cell(row=x, column=column_index_from_string("AE")).value
            # Increment row and column offsets
            #currentRow += 1
            columnOffset += 1
        currentRow += 1

#### Create and load workbooks 

In [257]:
#Open data workbook and access the sheet
dataWorkbook = openpyxl.load_workbook('data.xlsx')
dataSheet = dataWorkbook.get_sheet_by_name('Sheet1')
#Open workbook containing list of names
namesWorkbook = openpyxl.load_workbook('names.xlsx')
nameSheet = namesWorkbook.get_sheet_by_name('Sheet1')
#Create a blank workbook to store everything
reportWorkbook = openpyxl.Workbook()
reportSheet = reportWorkbook.get_sheet_by_name('Sheet')
reportWorkbook.save('reportTest.xlsx')

## Main program

In [258]:
generateReport()
reportWorkbook.save('reportTest.xlsx')